In [1]:
import os
import dask.dataframe as dd
import featuretools as ft
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
path_profile = r'D:\issp_data\zok_ssp_2017.csv'

log_root = r'D:\issp_data\fe_log_new'
out_root = r'D:\issp_data\features_total'

In [3]:
# convert to dummy variables
# 1.居住都道府県(参考)
# 2.居住エリア
# 3.年齢(5才刻み)
# 4.世代
# 5.性別
# 6.性年代
# 7.メディアターゲット区分
# 8.未既婚
# 9.職業(詳細)
# 10.職種
# 11.仕事環境
# 12.個人年収
# 13.1ヶ月平均小遣い
# 14.最終学歴 
# 15.家族人数
# 16.家族構成
# 17.世帯年収
# 18.家屋形態
# 19.生協加入有無
# 20.配偶者の同居有無
# 21.同居配偶者年齢(5才刻み)
# 22.同居配偶者職業
# 23.末子年齢
# 24.家族人員(17才以下男子)
# 25.家族人員(17才以下女子)
# 26.家族人員(0～2才(乳児))
# 27.家族人員(3～5才(就学前児童))
# 28.家族人員(6～11才(小学生))
# 29.家族人員(12～14才(中学生))
# 30.家族人員(15～17才(高校生))
# 31.家族人員(65才以上)
# 32.自動車(自家用車保有)有無
# 33.同居子供人数（17才以下）
# 34.同居子供人数（14才以下）

dropcols = [
    'モニターID', '暗号化ID(40桁)', '居住都道府県(参考)', '居住エリア', 
    '年齢(5才刻み)_名称', '年齢(10才刻み)', '年齢(10才刻み)_名称', '世代', 
    '性別_名称', '性年代', 'メディアターゲット区分', '未既婚_名称', '職業', '職業_名称', 
    '職業(詳細)', '職種', '仕事環境', '個人年収_名称', '1ヶ月平均小遣い_名称', 
    '最終学歴_名称', '家族人数_名称', '家族構成', '世帯年収_名称', '家屋形態',
    '生協加入有無_名称', '配偶者の同居有無', '同居配偶者年齢(5才刻み)_名称', 
    '同居配偶者年齢(10才刻み)', '同居配偶者年齢(10才刻み)_名称', '同居配偶者職業', 
    '末子年齢_名称', '家族人員(17才以下男子)_名称', '家族人員(17才以下女子)_名称', 
    '家族人員(0〜2才(乳児))_名称', '家族人員(3〜5才(就学前児童))_名称', 
    '家族人員(6〜11才(小学生))_名称', '家族人員(12〜14才(中学生))_名称', 
    '家族人員(15〜17才(高校生))_名称', '家族人員(65才以上)_名称', 
    '自動車(自家用車保有)有無_名称', '同居子供人数（17才以下）_名称', 
    '同居子供人数（14才以下）_名称', 'ウエイトセル', 'ウエイトセル_名称'
]

dummy = [
    '居住都道府県(参考)_名称',
    '居住エリア_名称',
    '世代_名称',  # 之所以讓這個本來可以catagorical的東西變成dummy, 是因為它跟年齡都是連續的, 有鑑於性質重複所以特別改了
    '性年代_名称',
    'メディアターゲット区分_名称',
    '職業(詳細)_名称',
    '職種_名称', 
    '仕事環境_名称',
    '家族構成_名称',
    '家屋形態_名称',
    '配偶者の同居有無_名称',
    '同居配偶者職業_名称'
]

columns_jp2en = {
    r'モニターID(11桁)': r'Monitor_ID'
}

df_dummy = pd.get_dummies(
                pd.read_csv(path_profile, encoding='shift-jis', 
                            usecols=dummy + ['モニターID(11桁)'])\
                    .rename(columns=columns_jp2en)\
                    .set_index('Monitor_ID'))

df_dummy.shape

(33932, 147)

In [4]:
# adjust offset of categorical variables
categorical = [
    '年齢(5才刻み)',  # 要減1, 調整offset --> F1
    '性別',  # 要減1, 其實這個categorical或dummy都可以, 因為是二元分類嘛, 但避免column數過多, 就變成cate vars --> F1
    '未既婚',  # 這也要減1, 理由同上 --> F1
    '個人年収',  # return 0 if x == 10 else x --> 如果 x == 10 反而是沒收入, 把它歸零, 其餘不變 --> F2
    '1ヶ月平均小遣い',  # return 0 if x == 10 else x - 1 --> 10是不明, 1是沒收入(要將它規0) --> F3
    
    '最終学歴',  # return 0 if x == 8 else x --> F4
    '家族人数',  # 不需要任何調整, good! --> None
    '世帯年収',  # return 0 if x == 6 else x --> F5
    '生協加入有無',  # 要減1, 調整offset --> F1
    '同居配偶者年齢(5才刻み)',  # return 0 if x == 10 else x, 因為x == 10表示沒有配偶/沒有同居 --> F2
    
    '末子年齢',  # 要減1, 調整offset, 因為x==1是表示沒小孩= = --> F1
    '家族人員(17才以下男子)',  # 要減1, 調整offset
    '家族人員(17才以下女子)',  # 同上
    '家族人員(0〜2才(乳児))',  # 同上
    '家族人員(3〜5才(就学前児童))',  # 同上
    
    '家族人員(6〜11才(小学生))',  # 同上
    '家族人員(12〜14才(中学生))',  # 同上
    '家族人員(15〜17才(高校生))',  # 同上
    '家族人員(65才以上)',  # 同上
    '自動車(自家用車保有)有無',  # 同上
    
    '同居子供人数（17才以下）',  # 同上, 要減1, 調整offset
    '同居子供人数（14才以下）'  # 同上, 要減1, 調整offset
]

F1 = lambda x: x - 1
F2 = lambda x: 0 if x == 10 else x
F3 = lambda x: 0 if x == 10 else x - 1
F4 = lambda x: 0 if x ==  8 else x
F5 = lambda x: 0 if x ==  6 else x

func_list = [F1, F1, F1, F2, F3, 
             F4, None, F5, F1, F2, 
             F1, F1, F1, F1, F1, 
             F1, F1, F1, F1, F1,
             F1, F1]

df_cate = pd.read_csv(path_profile, encoding='shift-jis', 
                      usecols=categorical + ['モニターID(11桁)'])\
                .rename(columns=columns_jp2en)\
                .set_index('Monitor_ID')

for col_nm, func_nm in zip(categorical, func_list):
    if func_nm:
        df_cate[col_nm] = df_cate[col_nm].map(func_nm)
    
df_profile = df_cate.join(df_dummy, on='Monitor_ID')

print(df_cate.shape)
print(df_profile.shape)
df_profile

(33932, 22)
(33932, 169)


,年齢(5才刻み),性別,未既婚,個人年収,1ヶ月平均小遣い,最終学歴,家族人数,世帯年収,生協加入有無,同居配偶者年齢(5才刻み),...,同居配偶者職業_名称_内職,同居配偶者職業_名称_学生,同居配偶者職業_名称_専業主婦･専業主夫,同居配偶者職業_名称_正社員(管理職),同居配偶者職業_名称_正社員(管理職以外の正社員),同居配偶者職業_名称_派遣社員・契約社員,同居配偶者職業_名称_無職,同居配偶者職業_名称_自営手伝い・家業手伝い,同居配偶者職業_名称_自営業・個人事業主,同居配偶者職業_名称_配偶者なし・非同居・不明
Monitor_ID,,,,,,,,,,,,,,,,,,,,,
50000203856,5,0,1,7,3,4,1,3,1,0,...,0,0,0,0,0,0,0,0,0,1
50000300858,9,1,0,0,3,3,6,5,0,9,...,0,0,0,0,0,1,0,0,0,0
50000448787,5,1,0,1,2,2,3,2,1,6,...,0,0,0,0,1,0,0,0,0,0
50000193009,4,0,0,5,2,4,4,2,1,4,...,0,0,0,0,0,0,0,0,0,0
50000300091,9,1,0,0,2,2,2,1,0,9,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50001973470,0,1,1,0,0,0,6,0,1,0,...,0,0,0,0,0,0,0,0,0,1
50001973824,2,0,1,0,3,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,1
50001975048,2,1,1,0,3,0,4,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
file_ls = sorted(os.listdir(log_root))

for month in range(12):
    path_os = os.path.join(log_root, file_ls[month * 4 + 3])
    df_os = dd.read_csv(path_os, compression='zip')\
                .set_index('Monitor_ID')
#     df_os裡面的dummy variable, iPhone==0, Android==1
    print(path_os, df_os.shape)
    
    for cata in range(3):
        file_nm = file_ls[month * 4 + cata]
        path_login = os.path.join(log_root, file_nm)
        file_nm = file_nm[:-9] + r'_features.csv'
        
        print(path_login, r'computing now ...')
        
        dd.read_csv(path_login, compression='zip')\
            .drop(columns='Date')\
            .join(df_os, on='Monitor_ID')\
            .join(df_profile, on='Monitor_ID')\
            .compute()\
            .to_csv(os.path.join(out_root, file_nm), 
                    header=True, index=None, compression='zip')
        
        print(f'>> {file_nm} save file completed!')

D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_os.csv (Delayed('int-f68b8407-512f-4b7a-b5bf-615781b1ba62'), 1)
D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_0logfe.csv computing now ...
>> s35809_MobileApp_201701_104628_0_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_1logfe.csv computing now ...
>> s35809_MobileApp_201701_104628_1_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201701_104628_2logfe.csv computing now ...
>> s35809_MobileApp_201701_104628_2_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201702_104645_os.csv (Delayed('int-4581b7a2-1f2e-443b-83fc-c36db95957f4'), 1)
D:\issp_data\fe_log_new\s35809_MobileApp_201702_104645_0logfe.csv computing now ...
>> s35809_MobileApp_201702_104645_0_features.csv save file completed!
D:\issp_data\fe_log_new\s35809_MobileApp_201702_104645_1logfe.csv computing now ...
>> s35809_MobileApp_201702_104645_1_features.csv save file com

In [6]:
path = r'D:\issp_data\features_total\s35809_MobileApp_201701_104628_0_features.csv'
df_os = dd.read_csv(path, compression='zip').compute()

df_os

,Monitor_ID,Day,Month,Weekday,ts_max,ts_mean,ts_median,ts_min,ts_std,ts_var,...,同居配偶者職業_名称_内職,同居配偶者職業_名称_学生,同居配偶者職業_名称_専業主婦･専業主夫,同居配偶者職業_名称_正社員(管理職),同居配偶者職業_名称_正社員(管理職以外の正社員),同居配偶者職業_名称_派遣社員・契約社員,同居配偶者職業_名称_無職,同居配偶者職業_名称_自営手伝い・家業手伝い,同居配偶者職業_名称_自営業・個人事業主,同居配偶者職業_名称_配偶者なし・非同居・不明
0,50000010367,5,1,3,1483658572,1.483615e+09,1.483618e+09,1483577852,26569.735436,7.059508e+08,...,0,0,0,0,0,0,0,0,0,0
1,50000011091,2,1,0,1483348718,1.483349e+09,1.483349e+09,1483348602,40.840135,1.667917e+03,...,0,0,0,0,1,0,0,0,0,0
2,50000011091,16,1,0,1484552063,1.484552e+09,1.484552e+09,1484551003,358.379464,1.284358e+05,...,0,0,0,0,1,0,0,0,0,0
3,50000011730,25,1,2,1485381088,1.485360e+09,1.485370e+09,1485329765,19049.690746,3.628907e+08,...,0,0,0,0,0,0,0,0,1,0
4,50000011763,23,1,0,1485209843,1.485188e+09,1.485185e+09,1485161078,15021.548332,2.256469e+08,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316472,50001841730,5,1,3,1483651711,1.483615e+09,1.483616e+09,1483577863,22218.174164,4.936473e+08,...,0,0,0,0,0,0,0,0,1,0
316473,50001841970,29,1,6,1485729375,1.485695e+09,1.485690e+09,1485674463,14757.261257,2.177768e+08,...,0,0,0,0,0,0,0,0,0,1
316474,50001842358,23,1,0,1485209017,1.485176e+09,1.485178e+09,1485150603,18608.005071,3.462579e+08,...,0,0,0,0,1,0,0,0,0,0
316475,50001842551,23,1,0,1485208600,1.485179e+09,1.485186e+09,1485143727,23591.244338,5.565468e+08,...,0,0,0,0,0,0,0,0,0,1
